In [2]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [6]:
import json

results = []

for i in range(4):
    with open(f"cleanup_results_new_32_shots_{i}_g2_big_higher_l1.jsonl") as f:
        lines = f.readlines()
        lines = lines[len(lines) // 2:]
        results.extend([json.loads(line) for line in lines])

In [7]:
print(len(results))

23


In [8]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [9]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf", from_type="gemma2", load_eager=True, device_map=f"tpu:0")

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [11]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [12]:
# task_names = ["en_es", "antonyms", "person_profession", "es_en", "present_simple_gerund", "present_simple_past_simple", "person_profession", "person_language", "country_capital", "football_player_position"]
# task_name = task_names[1]
task_names = list(tasks.keys())

In [13]:
import jax.numpy as jnp
import jax


from sprint.task_vector_utils import ICLRunner, logprob_loss, get_tv, make_act_adder
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper


get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)



def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs


In [14]:
from sprint.icl_sfc_utils import AblatedModule
layer = 16
mask_name = "arrow"

In [15]:
from micrlhf.utils.load_sae import get_dm_res_sae, sae_encode


sae = get_dm_res_sae(layer=layer, load_65k=False)

google/gemma-scope-2b-pt-res/layer_16/width_16k/average_l0_78/params.npz


In [16]:
import numpy as np

features = []
task_features = {}
tv_features = {}

for task_name in task_names:
    task_results = [result for result in results if result["task"] == task_name and result["layer"] == layer]
    
    for result in task_results:
        weights = np.array(result["weights"])
        tv = np.array(result["tv"])
        # s = jax.nn.softplus(sae["s_gate"]) * sae["scaling_factor"]
        # threshold = jnp.maximum(0, sae["b_gate"] - sae["b_enc"] * s)
        # w = weights
        # w = w * (w > 0)

        _, w, _ = sae_encode(sae, None, pre_relu=weights)

        _, w_tv, _ = sae_encode(sae, tv)
        # print(threshold)

        new_features = np.nonzero(w)[0].tolist()
        features += new_features
        print(task_name, "TVC:", result["loss"], "TV:", result["tv_loss"], new_features)

        task_features[task_name] = new_features
        tv_features[task_name] = np.nonzero(w_tv)[0].tolist()
    

features = list(set(features))

len(features)

location_continent TVC: 6.625 TV: 5.8125 [7107, 7464, 8646]
football_player_position TVC: 8.6875 TV: 7.21875 [817, 10300, 12944]
location_religion TVC: 7.21875 TV: 5.0625 [7107]
location_language TVC: 2.921875 TV: 4.75 [2707, 7107, 13804]
person_profession TVC: 5.84375 TV: 5.8125 [817, 4012, 7107, 10300, 10466, 12944]
location_country TVC: 3.53125 TV: 3.8125 [7107, 12540]
country_capital TVC: 0.87890625 TV: 2.46875 [16315]
person_language TVC: 2.546875 TV: 3.265625 [2707, 7107, 13804]
singular_plural TVC: 2.875 TV: 2.9375 [4285, 12777]
present_simple_past_simple TVC: 3.875 TV: 4.34375 [5496, 5774, 12721, 12777]
antonyms TVC: 2.625 TV: 2.421875 [4820, 8941, 13692, 16222]
plural_singular TVC: 2.375 TV: 1.703125 [12777, 15358]
present_simple_past_perfect TVC: 4.90625 TV: 4.65625 [12777]
present_simple_gerund TVC: 2.890625 TV: 3.859375 [4592, 12721, 12777, 13623]
en_it TVC: 4.3125 TV: 8.9375 [2683, 4766, 6628, 12647, 13976, 15511, 16176]
it_en TVC: 2.6875 TV: 2.40625 [493, 4766]
en_fr TVC:

49

In [18]:
for task_name in task_names:
    print(task_name, len(task_features[task_name]), len(tv_features[task_name]))

location_continent 3 35
football_player_position 3 39
location_religion 1 33
location_language 3 34
person_profession 6 36
location_country 2 23
country_capital 1 29
person_language 3 30
singular_plural 2 45
present_simple_past_simple 4 52
antonyms 4 50
plural_singular 2 43
present_simple_past_perfect 1 47
present_simple_gerund 4 52
en_it 7 39
it_en 2 47
en_fr 7 38
en_es 5 40
fr_en 1 50
es_en 2 47
algo_last 4 44
algo_first 5 47
algo_second 5 48


In [19]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector

task_losses_positive = {}

n_few_shots, batch_size, max_seq_len = 20, 32, 64
seed = 10

prompt = "Follow the pattern:\n{}"

def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(task_names):

    sep = 3978
    pad = 0


    pairs = list(tasks[task_name].items())

    n_shot = n_few_shots - 1
    if task_name.startswith("algo"):
        n_shot = 8

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    scale = 60

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]

    
    def steer_with_direction(direction):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)
        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2, do_ppl=False)

    task_losses_positive[task_name] = [[steer_with_direction(sae["W_dec"][feature]).tolist() for feature in tqdm(features)]]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    task_losses_positive[task_name].append(logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2, do_ppl=False).tolist())

  0%|          | 0/23 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = np.minimum(losses, base_loss)

    losses = (base_loss - losses) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    # losses = (losses - min_loss) / (max_loss - min_loss)

    # losses = losses / max_loss

    # losses[losses < 0.2] = 0.0
    # losses[losses > 1.0] = 1.0

    # losses = np.clip(losses, 0.2, 1.0)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

def make_heatmap():
    normalized_losses = {}

    drop_features = []
    drop_ids = [features.index(feature) for feature in drop_features]

    features_dropped = [feature for feature in features if feature not in drop_features]

    for task_name, losses in task_losses_positive.items():
        base_loss = losses[1]
        losses = losses[0]
        losses = np.array(losses)
        losses = np.delete(losses, drop_ids)

        losses = np.minimum(losses, base_loss)

        losses = (base_loss - losses) / base_loss
        
        max_loss = np.max(losses)
        # min_loss = np.min(losses)

        losses = losses / max_loss

        losses[losses < 0.2] = 0.0
        losses[losses > 1.0] = 1.0
        
        # mean_loss = np.mean(losses - base_loss)

        # base_acc = losses[1][1]

        # accs = [base_acc - loss[1] for loss in losses[0]]
        normalized_losses[task_name] = losses


    heatmap = np.zeros((len(task_names), len(features_dropped)))

    for i, task_name in enumerate(task_names):
        for j, feature in enumerate(features_dropped):
            heatmap[i, j] = normalized_losses[task_name][j]

    return heatmap


# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.log(1 + heatmap * 40)

# heatmap = np.clip(heatmap, -5, 5)

# avg_heatmap = np.sum(heatmap != 1, axis=0)

max_features = 100

sort_map = make_heatmap()
# sort_map = heatmap

avg_heatmap  = np.max(sort_map, axis=0)

sorted_idx = np.argsort(-avg_heatmap)

heatmap = heatmap[:, sorted_idx]

sort_map = sort_map[:, sorted_idx]

# min_heatmap = np.min(heatmap, axis=1)   

min_pos = np.argmax(sort_map, axis=1)

y_sorted_idx = np.argsort(min_pos)

heatmap = heatmap[y_sorted_idx]


heatmap = np.flip(heatmap, axis=0)

fig = px.imshow(heatmap[:, :max_features], x=[str(features_dropped[x]) for x in sorted_idx][:max_features], y=[task_names[x] for x in y_sorted_idx][::-1], width=2000, height=600, aspect="auto", color_continuous_scale="Blues")

def make_filp_mask():
    heatmap = np.zeros((len(task_names), len(features)))
    feat_to_idx = {feat: i for i, feat in enumerate(features)}


    for i, task_name in enumerate(task_names):
        feats = tv_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = 2


        feats = task_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = 1

    heatmap = heatmap[y_sorted_idx]
    heatmap = heatmap[:, sorted_idx]

    return heatmap

filp_mask = make_filp_mask()


filp_mask = np.flip(filp_mask, axis=0)[:, :max_features]

annotation_style = dict(
    showarrow=False,
    font=dict(color="black", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["🟢"] for x in np.argwhere(filp_mask == 1)
}

special_cells.update({
    tuple(x): ["🔴"] for x in np.argwhere(filp_mask == 2)
})

# special_cells.update({
#     tuple(x): ["✔", "*"] for x in np.argwhere(filp_mask == 3)
# })

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]
fig.update_layout(annotations=annotations)

fig.show()


In [15]:
fig = px.imshow(heatmap[:, :max_features], x=[str(features_dropped[x]) for x in sorted_idx][:max_features], y=[task_names[x] for x in y_sorted_idx][::-1], width=2000, height=600, aspect="auto", color_continuous_scale="Blues", labels=dict(x="Feature", y="Task", color="Relative loss decrease"))

annotation_style = dict(
    showarrow=False,
    font=dict(color="MediumSeaGreen", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 1)
}

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

annotation_style = dict(
    showarrow=False,
    font=dict(color="red", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells ={
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 2)
}

# special_cells.update({
#     tuple(x): ["✔", "*"] for x in np.argwhere(filp_mask == 3)
# })


annotations += [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

# legend_annotations = [
#     dict(
#         x=1.05, y=0.8, text="🟢 Feature is present after cleaning", showarrow=False,
#         font=dict(size=8), xanchor="left", yanchor="top"
#     ),
#     dict(
#         x=1.05, y=0.95, text="🔴 Feature is in task vector", showarrow=False,
#         font=dict(size=8), xanchor="left", yanchor="top"
#     )
# ]

# fig.update_layout(width =600, height=250, 
#                 font_family="Serif", font_size=7, 
#                 margin_l=5, margin_t=5, margin_b=5, margin_r=5,
#         title=dict(
#         text=legend_html,
#         x=0.525,  # Center the title
#         y=0.09,  # Place the legend below the plot
#         xanchor="center",
#         yanchor="top"
#     ))


# legend_html = """
# <span style="font-size:8px;">
# <span style="color:red">•</span> Feature is in task vector | <span style="color:green">•</span> Feature is present after cleaning
# </span>
# """

# Update layout with annotations
fig.update_layout(annotations=annotations)

legend_html = """
<span style="font-size:8px;">
<span style="color:red">●</span> Feature is in task vector | <span style="color:MediumSeaGreen">●</span> Feature is present after cleaning
</span>
"""

fig.update_layout(width =1200, height=300, 
                font_family="Serif", font_size=7,
                margin_l=5, margin_t=5, margin_b=5, margin_r=5,
        title=dict(
        text=legend_html,
        x=0.525,  # Center the title
        y=0.079,  # Place the legend below the plot
        xanchor="center",
        yanchor="top"
    ))


# fig.update_layout(annotations=annotations)

import plotly.io as pio
# pio.write_image(fig, "micrlhf-progress/images/executor_heatmap_l12_new_gemma.pdf", width =1200, height=300)
fig

In [42]:
fig = px.imshow(heatmap[:, :23], x=[str(features_dropped[x]) for x in sorted_idx[:23]], y=[task_names[x] for x in y_sorted_idx][::-1], width=800, height=600, aspect="auto", color_continuous_scale="Blues", labels=dict(x="Feature", y="Task", color="Effect strength"))

annotation_style = dict(
    showarrow=False,
    font=dict(color="MediumSeaGreen", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 1)
}

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

annotation_style = dict(
    showarrow=False,
    font=dict(color="red", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells ={
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 2)
}

annotations += [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]
fig.update_layout(width =400, height=300, 
                font_family="Serif", font_size=7,
                margin_l=5, margin_t=5, margin_b=5, margin_r=5,
        )


# fig.update_layout(annotations=annotations)

import plotly.io as pio
pio.write_image(fig, "micrlhf-progress/images/executor_heatmap_l12_new.pdf", width =400, height=300)
fig

In [20]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector, add_vector_many

positive_task_losses_testing = {}

n_few_shots, batch_size, max_seq_len = 1, 16, 64
seed = 10

prompt = "Follow the pattern:\n{}"


def calc_acc(tokens, sep, logits):
    preds = logits.argmax(-1)
    preds = preds[:, :-1]
    targets = tokens[:, 1:]

    mask = tokens[:, :-1] == sep

    hits = targets == preds

    hits = hits * mask

    hits = hits.sum()
    return hits / mask.sum()




def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(task_names):

    sep = 3978
    pad = 0
    pairs = list(tasks[task_name].items())

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]

    def steer_with_direction(direction, scale):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)

        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        acc = calc_acc(train_tokens, sep, logits)

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2), acc

    # _features = task_features[task_name]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    acc = calc_acc(train_tokens, sep, logits)


    for feature in tqdm(features):
        positive_task_losses_testing[(task_name, feature)] = [[steer_with_direction(sae["W_dec"][feature], scale) for scale in np.logspace(1.5, 2.5, 15)]]
        positive_task_losses_testing[(task_name, feature)].append((logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2), acc))

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


  0%|          | 0/49 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


...

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


  0%|          | 0/49 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


  0%|          | 0/49 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [23]:


feature = 13692

heatmap = np.zeros((len(task_names), 15))

for i, task_name in enumerate(task_names):
    losses, base_loss = positive_task_losses_testing[(task_name, feature)]
    # print(task_name, base_loss[0])
    if task_name.startswith("algo"):
        continue
    # if base_loss[1] < 0.4:
    #     continue
    # negative_task_losses_testing[(task_name, feature)]
    heatmap[i] =  float(base_loss[0]) - np.array([float(loss) for loss, _  in losses])
    heatmap[i] = heatmap[i] / float(base_loss[0])
    # heatmap[i] = heatmap[i] / float(base_loss[1])
# clipped = np.maximum(heatmap, 0)
# heatmap /= clipped.mean(axis=0)
# print(clipped.mean(0).shape)

px.imshow(heatmap, x=np.logspace(1.5, 2.5, 15), y=task_names, height=600)

In [23]:
import plotly.express as px
max_features = 100

heatmap = np.zeros((len(task_names), len(features)))    

for i, feature in enumerate(features):
    run_data = np.zeros((len(task_names), 15))
    for j, task_name in enumerate(task_names):
        losses, base_loss = positive_task_losses_testing[(task_name, feature)]
        n_loss = 0
        # if task_name.startswith("algo"):
        #     continue
        # if base_loss[1] < 0.3:
        #     continue
        # if task_name == "present_simple_gerund":
        #     continue
        run_data[j] = float(base_loss[n_loss]) - np.array([float(loss[n_loss]) for loss  in losses])
        # print(
        #     task_name, base_loss
        # )
        run_data[j] = run_data[j] / float(base_loss[n_loss])
        # run_data[j] *= -1
    # run_data = run_data.max(0)
    # find first idx where > 1
    first_idx = np.argmax(run_data.max(0))

    # print(np.logspace(1.5, 2.5, 15)[first_idx])
    
    # run_data = np.maximum(run_data, 0)
    run_data[run_data < 0] = 0
    # run_data[run_data > 0.1] = 0.3
    heatmap[:, i] = run_data[:, first_idx]
    # heatmap[:, i] = run_data[:, first_idx]
    # heatmap[:, i] = heatmap[:, i] / heatmap[:, i].mean() 

# heatmap = heatmap / heatmap.mean(0)


# heatmap = np.maximum(heatmap, 0)
def make_sortmap():
    heatmap = np.zeros((len(task_names), len(features)))    

    for i, feature in enumerate(features):
        run_data = np.zeros((len(task_names), 15))
        for j, task_name in enumerate(task_names):
            losses, base_loss = positive_task_losses_testing[(task_name, feature)]
            n_loss = 0
            # if task_name.startswith("algo"):
            #     continue
            # if base_loss[1] < 0.3:
            #     continue
            # if task_name == "present_simple_gerund":
            #     continue
            run_data[j] = float(base_loss[n_loss]) - np.array([float(loss[n_loss]) for loss  in losses])
            # print(
            #     task_name, base_loss
            # )
            run_data[j] = run_data[j] / float(base_loss[n_loss])
            # run_data[j] *= -1
        # run_data = run_data.max(0)
        # find first idx where > 1
        first_idx = np.argmax(run_data.max(0))

        # print(first_idx)
        
        # run_data = np.maximum(run_data, 0)
        run_data[run_data < 0] = 0
        # run_data[run_data > 0.1] = 0.3
        heatmap[:, i] = run_data[:, first_idx]
        # heatmap[:, i] = run_data[:, first_idx]
        # heatmap[:, i] = heatmap[:, i] / heatmap[:, i].mean() 

    heatmap = heatmap / heatmap.max(axis=1, keepdims=True)

    return heatmap

sortmap = make_sortmap()

# heatmap = np.maximum(heatmap, 0)
sorted_indices = np.argsort((sortmap ).max(0))

sorted_indices = sorted_indices[::-1]

sortmap = sortmap[:, sorted_indices]

min_pos = np.argmax(sortmap, axis=1)

y_sorted_idx = np.argsort(min_pos)

heatmap = heatmap[:, sorted_indices]
heatmap = heatmap[y_sorted_idx]

heatmap = np.flip(heatmap, axis=0)


def make_filp_mask():
    heatmap = np.zeros((len(task_names), len(features)))
    feat_to_idx = {feat: i for i, feat in enumerate(features)}


    for i, task_name in enumerate(task_names):
        feats = tv_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = 2


        feats = task_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = 1

    heatmap = heatmap[y_sorted_idx]
    heatmap = heatmap[:, sorted_indices]

    return heatmap

filp_mask = make_filp_mask()


filp_mask = np.flip(filp_mask, axis=0)[:, :max_features]

annotation_style = dict(
    showarrow=False,
    font=dict(color="black", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["🟢"] for x in np.argwhere(filp_mask == 1)
}

special_cells.update({
    tuple(x): ["🔴"] for x in np.argwhere(filp_mask == 2)
})

# special_cells.update({
#     tuple(x): ["✔", "*"] for x in np.argwhere(filp_mask == 3)
# })

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

fig = px.imshow(heatmap, x=[str(features[i]) for i in sorted_indices] ,  y=[task_names[i] for i in y_sorted_idx][::-1] , height=600, width=2000, color_continuous_scale="Blues")


fig.update_layout(annotations=annotations)

# special_cells = {
#     tuple(x): ["🔴"] for x in np.argwhere(filp_mask == -1)
# }

# for cell, icons in special_cells.items():
#     for i, icon in enumerate(icons):
#         fig.add_annotation(
#             x=cell[1],  # Column index
#             y=cell[0],  # Row index
#             text=icon,  # Icon or text
#             showarrow=False,
#             font=dict(color="white", size=5),
#             xanchor="center",
#             yanchor="middle",
#             yshift=i * 10  # Adjust vertical position to stack icons
        # )

fig


In [28]:
fig = px.imshow(heatmap[:, :max_features], x=[str(features[x]) for x in sorted_indices][:max_features], y=[task_names[x] for x in y_sorted_idx][::-1], width=2000, height=600, aspect="auto", color_continuous_scale="Blues", labels=dict(x="Feature", y="Task", color="Relative loss decrease"))

annotation_style = dict(
    showarrow=False,
    font=dict(color="MediumSeaGreen", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 1)
}

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

annotation_style = dict(
    showarrow=False,
    font=dict(color="red", size=7),
    xanchor="center",
    yanchor="middle"
)

special_cells ={
    tuple(x): ["●"] for x in np.argwhere(filp_mask == 2)
}

# special_cells.update({
#     tuple(x): ["✔", "*"] for x in np.argwhere(filp_mask == 3)
# })


annotations += [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, xshift=0, yshift=0 - i * 6)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]

# legend_annotations = [
#     dict(
#         x=1.05, y=0.8, text="🟢 Feature is present after cleaning", showarrow=False,
#         font=dict(size=8), xanchor="left", yanchor="top"
#     ),
#     dict(
#         x=1.05, y=0.95, text="🔴 Feature is in task vector", showarrow=False,
#         font=dict(size=8), xanchor="left", yanchor="top"
#     )
# ]

# fig.update_layout(width =600, height=250, 
#                 font_family="Serif", font_size=7, 
#                 margin_l=5, margin_t=5, margin_b=5, margin_r=5,
#         title=dict(
#         text=legend_html,
#         x=0.525,  # Center the title
#         y=0.09,  # Place the legend below the plot
#         xanchor="center",
#         yanchor="top"
#     ))


# legend_html = """
# <span style="font-size:8px;">
# <span style="color:red">•</span> Feature is in task vector | <span style="color:green">•</span> Feature is present after cleaning
# </span>
# """

# Update layout with annotations
fig.update_layout(annotations=annotations)

legend_html = """
<span style="font-size:8px;">
<span style="color:red">●</span> Feature is in task vector | <span style="color:MediumSeaGreen">●</span> Feature is present after cleaning
</span>
"""

fig.update_layout(width =700, height=300, 
                font_family="Serif", font_size=7,
                margin_l=5, margin_t=5, margin_b=5, margin_r=5,
        title=dict(
        text=legend_html,
        x=0.525,  # Center the title
        y=0.079,  # Place the legend below the plot
        xanchor="center",
        yanchor="top"
    ))


# fig.update_layout(annotations=annotations)

import plotly.io as pio
pio.write_image(fig, "micrlhf-progress/images/executor_heatmap_l12_new_gemma_2_16k.pdf", width =700, height=300)
fig

: 

In [19]:
with open("micrlhf-progress/executor_heatmap_l12.json", "w") as f:
    json.dump({"heatmap": heatmap.tolist(), "features": [features_dropped[x] for x in sorted_idx], "task_names": [task_names[x] for x in y_sorted_idx]}, f)

In [24]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = np.minimum(losses, base_loss)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

# avg_heatmap = np.sum(heatmap != 1, axis=0)

avg_heatmap  = np.mean(heatmap, axis=0)

sorted_idx = np.argsort(avg_heatmap)

heatmap = heatmap[:, sorted_idx]


fig = px.imshow(heatmap, x=[str(features_dropped[x]) for x in sorted_idx], y=task_names, width=1400, height=400, aspect="auto")

fig.show()


In [34]:
tvs = [x["tv"] for x in results if x["layer"] == layer]

len(tvs)

23

In [43]:
avg_tv = np.mean(tvs, axis=0)

avg_tv = avg_tv.astype(jnp.float32)

In [58]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector

task_losses_positive = {}

n_few_shots, batch_size, max_seq_len = 20, 16, 256
seed = 10

prompt = "Follow the pattern:\n{}"

def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(["location_religion"]):

    sep = 3978
    pad = 0


    pairs = list(tasks[task_name].items())

    n_shot = n_few_shots - 1
    if task_name.startswith("algo"):
        n_shot = 8

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    # scale = 25

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]
    
    feature = 11172

    def steer_with_direction(direction, scale):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale + avg_tv * 0.5
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)
        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2)

    task_losses_positive[task_name] = [[steer_with_direction(sae["W_dec"][feature], scale).tolist() for scale in tqdm(np.logspace(0, 2, 100))]]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    task_losses_positive[task_name].append(logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2).tolist())

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning:

scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.



In [59]:
px.line(x=np.logspace(0, 2, 100), y=task_losses_positive["location_religion"][0])

In [54]:
px.line(x=np.logspace(0, 2, 100), y=task_losses_positive["antonyms"][0])

In [47]:
task_losses_positive["antonyms"][0]

[6.28125,
 6.28125,
 6.25,
 6.25,
 6.25,
 6.25,
 6.21875,
 6.25,
 6.21875,
 6.21875,
 6.21875,
 6.1875,
 6.1875,
 6.1875,
 6.15625,
 6.125,
 6.125,
 6.125,
 6.125,
 6.09375,
 6.09375,
 6.0625,
 6.03125,
 6.0625,
 6.03125,
 6.0,
 5.96875,
 5.96875,
 5.9375,
 5.9375,
 5.90625,
 5.875,
 5.84375,
 5.8125,
 5.8125,
 5.78125,
 5.71875,
 5.65625,
 5.625,
 5.59375,
 5.5625,
 5.5,
 5.5,
 5.4375,
 5.40625,
 5.34375,
 5.28125,
 5.25,
 5.15625,
 5.09375,
 5.0625,
 4.96875,
 4.90625,
 4.8125,
 4.75,
 4.6875,
 4.59375,
 4.5,
 4.40625,
 4.3125,
 4.21875,
 4.09375,
 4.0,
 3.90625,
 3.8125,
 3.703125,
 3.671875,
 3.609375,
 3.578125,
 3.578125,
 3.640625,
 3.703125,
 3.796875,
 3.90625,
 4.03125,
 4.15625,
 4.3125,
 4.53125,
 4.6875,
 4.875,
 5.09375,
 5.28125,
 5.5625,
 5.8125,
 6.09375,
 6.375,
 6.625,
 6.90625,
 7.1875,
 7.5,
 7.75,
 8.0,
 8.3125,
 8.5625,
 8.8125,
 9.0,
 9.3125,
 9.5625,
 9.8125,
 10.0625]

In [46]:
heatmap.shape

(23, 129)

In [45]:
sorted_idx

array([[ 72, 119, 117, ...,  35,  67, 128],
       [ 13,  33,  69, ...,  44,  56, 128],
       [103, 118,  78, ...,  44,  42,  64],
       ...,
       [106,  27,  15, ...,  42,  37, 128],
       [ 89, 116,  73, ...,  34, 127, 128],
       [ 73,  39,   1, ...,  37,  68, 128]])

In [67]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in negative_task_losses.items():
    base_loss = losses[1]
    losses = [x[1].tolist() for x in losses[0]]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = (losses - base_loss[1]) 
    
    # max_loss = np.max(losses)
    # min_loss = np.min(losses)

    # losses = (losses - min_loss) / (max_loss - min_loss)

    # losses = (losses - min_loss) / (max_loss - min_loss)
    # loss = (base_loss - losses) / base_loss
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

std_heatmap = np.std(heatmap, axis=0)

sorted_idx = np.argsort(-std_heatmap)

heatmap = heatmap[:, sorted_idx]

labels = [str(features_dropped[x]) for x in sorted_idx]

fig = px.imshow(heatmap, x=labels, y=task_names, width=1400, height=800, aspect="auto")

fig.show()


In [66]:
task_features

{'location_continent': [6780, 7578, 9662, 14612, 24925],
 'football_player_position': [7491, 13458, 27401],
 'location_religion': [7578, 9600, 9662, 11172, 14612, 20832, 24925, 27401],
 'location_language': [7578, 9662, 11172, 24925],
 'person_profession': [6413, 7491, 7578, 13458, 27401],
 'location_country': [850, 9662, 11173, 24925],
 'country_capital': [11173, 17636, 18803],
 'person_language': [850, 7578, 11172, 13458, 24925],
 'singular_plural': [2930, 6594, 12943, 14612],
 'present_simple_past_simple': [2930, 6594, 15356],
 'antonyms': [7578, 7739, 10720, 11618, 19097, 19112, 25576],
 'plural_singular': [2930, 6594],
 'present_simple_past_perfect': [2930, 6594, 15356],
 'present_simple_gerund': [6594, 15554],
 'en_it': [2539, 5991, 6594, 26594],
 'it_en': [5579, 12943, 16490],
 'en_fr': [2539, 2930, 5991],
 'en_es': [5991, 6594, 26594, 26987],
 'fr_en': [5579, 12943, 16490],
 'es_en': [5579, 12943, 16490],
 'algo_last': [2930, 5991, 6594, 12943, 14612, 27268],
 'algo_first': [],
 'algo_second': [6594, 16996, 27268]}

In [120]:
features_dropped.index(7491)

69

In [121]:
mean_heatmap = np.mean(heatmap, axis=0)

drop_features = np.where(mean_heatmap < 0.9)[0]

drop_features = [features_dropped[x] for x in drop_features]

drop_features

[26127,
 6171,
 10787,
 30247,
 3113,
 9782,
 32320,
 15947,
 23682,
 12943,
 16016,
 13458,
 30888,
 19112,
 11491,
 23312,
 7967,
 7491,
 1878,
 20832,
 6503,
 26987,
 2939,
 31620,
 11172,
 25534,
 6594,
 5579,
 23514,
 6629,
 18416]

In [115]:
heatmap.shape

(23, 129)

In [116]:
(22 + 0.1)/23

0.9608695652173914

In [127]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)


    losses = np.minimum(losses, base_loss * 1.5)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / ((max_loss - min_loss) + 1e-6)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

heatmap = np.log(heatmap + 1)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names, color_continuous_scale='YlGnBu')

fig.show()

In [125]:
task_losses_positive["present_simple_past_perfect"]

[[10.3125,
  4.40625,
  4.75,
  6.6875,
  6.0625,
  5.15625,
  5.96875,
  5.9375,
  5.15625,
  7.8125,
  6.84375,
  4.75,
  7.125,
  7.625,
  6.25,
  4.875,
  6.5,
  5.1875,
  4.25,
  4.875,
  5.75,
  9.625,
  6.03125,
  10.5,
  6.6875,
  10.625,
  5.46875,
  9.1875,
  9.625,
  11.125,
  13.125,
  6.90625,
  4.21875,
  5.71875,
  4.75,
  4.71875,
  6.0625,
  5.625,
  5.03125,
  6.6875,
  4.96875,
  6.65625,
  11.1875,
  9.8125,
  7.375,
  4.875,
  10.125,
  6.75,
  7.96875,
  6.8125,
  5.625,
  14.75,
  13.1875,
  7.9375,
  5.28125,
  13.3125,
  6.3125,
  6.71875,
  5.1875,
  19.375,
  6.5,
  6.65625,
  5.25,
  4.6875,
  6.03125,
  7.9375,
  11.625,
  10.4375,
  6.5,
  6.09375,
  6.9375,
  10.0,
  17.0,
  5.15625,
  13.625,
  6.59375,
  5.1875,
  7.1875,
  5.34375,
  13.0,
  4.96875,
  7.3125,
  4.75,
  8.75,
  5.09375,
  5.0,
  17.875,
  6.09375,
  7.1875,
  5.34375,
  6.9375,
  8.5625,
  12.8125,
  5.0625,
  12.75,
  10.6875,
  9.0625,
  10.5,
  5.625,
  11.25,
  6.09375,
  7.1875,
  9.25,
  16.375,
  9.5625,
  5.78125,
  7.125,
  8.3125,
  10.0625,
  10.5625,
  5.125,
  3.53125,
  8.1875,
  6.28125,
  6.03125,
  11.875,
  6.59375,
  4.84375,
  8.875,
  6.1875,
  5.84375,
  5.03125,
  6.78125,
  6.25,
  4.25,
  8.8125,
  6.40625,
  8.375,
  4.0],
 3.84375]

In [84]:
mean_heatmap

array([0.10833333, 0.02105735, 0.0625    , 0.10611511, 0.0734767 ,
       0.07401033, 0.05617978, 0.05755396, 0.03819095, 0.03888889,
       0.19101124, 0.05395683, 0.22682446, 0.        , 0.06231156,
       0.0242411 , 0.10215054, 0.10453649, 0.04536489, 0.0094086 ,
       0.09346734, 0.13169643, 0.07777778, 0.22767857, 0.0915047 ,
       0.225     , 0.00416667, 0.00414938, 0.13403509, 0.2040201 ,
       0.27482014, 0.        , 0.        , 0.07482014, 0.06797853,
       0.05241935, 0.10663082, 0.01258993, 0.04496403, 0.02242991,
       0.04120603, 0.06919643, 0.39568345, 0.01032702, 0.02329749,
       0.08481262, 0.00280407, 0.09849246, 0.10555556, 0.04676259,
       0.01798561, 0.15654952, 0.26517572, 0.20143885, 0.07194245,
       0.617357  , 0.10267857, 0.        , 0.05527638, 0.15833333,
       0.13169643, 0.        , 0.01798561, 0.0729783 , 0.15779093,
       0.03718638, 0.03472222, 0.26111111, 0.13824561, 0.        ,
       0.10653266, 0.32070175, 0.        , 0.        , 0.2       ,
       0.02284946, 0.04856115, 0.09444444, 0.        , 0.4       ,
       0.08140704, 0.15107914, 0.03888889, 0.        , 0.08243728,
       0.00703518, 0.1522807 , 0.13632653, 0.04723618, 0.        ,
       0.04444444, 0.28801431, 0.375     , 0.06834532, 0.10286225,
       0.36628111, 0.12777778, 0.12096774, 0.01299283, 0.1       ,
       0.15770609, 0.16944444, 0.36094675, 0.        , 0.        ,
       0.03125   , 0.        , 0.17741935, 0.21111111, 0.11157895,
       0.        , 0.        , 0.13258786, 0.02817209, 0.05737705,
       0.47406082, 0.0031362 , 0.00277778, 0.00479233, 0.00138889,
       0.11079137, 0.01478495, 0.16853933, 0.02688172, 0.        ,
       0.07083333, 0.07222222, 0.05591054, 0.        ])

In [71]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in negative_task_losses.items():
    base_loss = losses[1]
    losses = [x[0].tolist() for x in losses[0]]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = losses - base_loss[0]
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names)

fig.show()


NameError: name 'negative_task_losses' is not defined